In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv

In [3]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [4]:
url = 'https://www.avito.ma/fr/maroc/appartements-%C3%A0_vendre?o='
options = webdriver.ChromeOptions()
options.add_argument('--inconginto')
driver = webdriver.Chrome(options=options)

path_to_file = "Appartements6.csv"
csvFile = open(path_to_file, 'w', newline='', encoding="utf-8")
csvWriter = csv.writer(csvFile)

csvWriter.writerow(["Titre", "Prix", "Ville" , "Type", "Secteur", "Etage", "Age du bien", "Surface habitable", "NB Salons", "NB Chambres", "NB Salles de Bain", "Frais de syndic / mois", "Description"])

## Boucle 1 permet de parcourir 100 page chaque page a 35 appartement
for i in range(800, 960):
    
    driver.get(url+str(i))

    try:
        app_link = driver.find_elements(By.CSS_SELECTOR, "div.sc-1nre5ec-1 > a".format(10))

        original_window = driver.current_window_handle

        j = 0
        for i in app_link:


            driver.execute_script("window.open('{}')".format(i.get_attribute("href")))
            driver.switch_to.window(driver.window_handles[1])

            #Recupperer Titre Annonce
            titre = ""
            try:
                titre = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-9 > h1.sc-1g3sn3w-12".format(10)).text
            except NoSuchElementException:
                titre = ""

            #Recupperer Prix
            prix = ""
            try:
                prix = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-10 > p.sc-1x0vz2r-0".format(10)).text
            except NoSuchElementException:
                prix = ""

            #Recupperer Ville
            ville = ""
            try:
                ville = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-7 > span.sc-1x0vz2r-0".format(10)).text
            except NoSuchElementException:
                ville = ""


            #Recupperer liste elements

            elements_dict = {
                "Type": "",
                "Secteur": "",
                "Étage": "",
                "Frais": "",
                "Salons": "",
                "Âge": "",
                "Salons":"",
                "Surface": "",
                "SalleDeBainTitleID": "",
                "ChambresTitleID": ""
            }
#             elements = []
#             try:
#                 elements = driver.find_elements(By.CSS_SELECTOR, "ol.sc-qmn92k-3 > li.sc-qmn92k-1 > span.sc-1x0vz2r-0".format(10))
#                 for key in elements_dict:
#                     for i in range(0,len(elements)):
#                         if elements[i].text.startswith(key):
#                             elements_dict[key] = elements[i+1].text
#                             break  # Exit the loop once a matching element is found
#             except NoSuchElementException:
#                 elements = []

#             icons = []
#             values = []
#             try:
#                 icons = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > div.sc-wdregf-0 > svg.av-icon".format(10))
#                 values = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > span.kQHNss".format(10))

#                 for key in elements_dict:
#                     for i in range(0,len(icons)):
#                         if icons[i].get_attribute("aria-labelledby").startswith(key):
#                             elements_dict[key] = values[i].text
#                             break  # Exit the loop once a matching element is found
#             except NoSuchElementException:
#                 icons = []
#                 alues = []

            elements = []
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, "ol.sc-qmn92k-3 > li.sc-qmn92k-1 > span.sc-1x0vz2r-0")
                elements_text = [element.text for element in elements]
                for key, value in elements_dict.items():
                    for i, element_text in enumerate(elements_text):
                        if element_text.startswith(key):
                            elements_dict[key] = elements_text[i + 1]
                            break  # Exit the loop once a matching element is found
            except NoSuchElementException:
                elements = []

            icons = []
            values = []

            try:
                icons = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > div.sc-wdregf-0 > svg.av-icon")
                values = driver.find_elements(By.CSS_SELECTOR, "div.sc-6p5md9-2 > span.kQHNss")

                icons_attributes = [element.get_attribute("aria-labelledby") for element in icons]

                for key, value in elements_dict.items():
                    for i, icon_attribute in enumerate(icons_attributes):
                        if icon_attribute.startswith(key):
                            elements_dict[key] = values[i].text
                            break  # Exit the loop once a matching element is found

            except NoSuchElementException:
                icons = []
                values = []


            #Recupperer Description
            description = ""
            try:
                description = driver.find_element(By.CSS_SELECTOR, "div.sc-1g3sn3w-16 > p.sc-ij98yj-0".format(10)).text
            except NoSuchElementException:
                description = ""

             #ecrire dans un fichier csv
            csvWriter.writerow([titre, prix, ville, elements_dict["Type"], elements_dict["Secteur"], elements_dict["Étage"], elements_dict["Âge"], elements_dict["Surface"], elements_dict["Salons"], elements_dict["ChambresTitleID"], elements_dict["SalleDeBainTitleID"], elements_dict["Frais"], description ])
            j += 1
            driver.close()
            driver.switch_to.window(original_window)
    except StaleElementReferenceException:
        next = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-2y0ggl-0 > a.sc-1cf7u6r-0")))
        #driver.get(url+str(i))
        

    

In [5]:
import pandas as pd

df2 = pd.read_csv("Appartements5.csv")
df3 = pd.read_csv("Appartements6.csv")


In [7]:
df3.to_csv('Appartements.csv', mode='a', index=False, header=False)

In [4]:
df = pd.read_csv("Appartements.csv")

In [5]:
df

,Titre,Prix,Ville,Type,Secteur,Etage,Age du bien,Surface habitable,NB Salons,NB Chambres,NB Salles de Bain,Frais de syndic / mois,Description
0,Appartement 165m² narjiss à El Jadida,1 160 000 DH,El Jadida,"Appartements, à vendre",Narjiss,1,NaN,229,3,6,2,NaN,un appart 115m2 et garçonnière 50m2 à vendre\n...
1,Appartement à vendre 62 m² à El Jadida,360 000 DH,El Jadida,"Appartements, à vendre",Autre secteur,2,6-10 ans,62,1,2,1,NaN,un appart 62m2 à vendre\nappart au quartier hi...
2,Appartement hay essalam 92 m² à El Jadida,750 000 DH,El Jadida,"Appartements, à vendre",Hay Essalam,2,6-10 ans,92,1,3,1,NaN,un appart 92m2 à vendre\nappart au quartier sa...
3,Appartement 110m² 2ème étg. résd. fermée Corniche,2 100 000 DH,Casablanca,"Appartements, à vendre",Bourgogne,2,1-5 ans,136,1,2,2,500.0,Appartement moderne 110m² au 2ème étg. en rési...
4,app a vendre,7 400 000 DH,El Mansouria,"Appartements, à vendre",Toute la ville,2,Neuf,60,1,1,1,450.0,vendre un belle appartement vie sur piscine et...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33316,Appartement 75m² en Vente à Larache,360 000 DH,Larache,"Appartements, à vendre",Toute la ville,1,1-5 ans,75,1,2,1,180.0,Appartement en vente\n\nPrix 36 malion
33317,Appartement en Vente à Farah Agadir,350 000 DH,Agadir,"Appartements, à vendre",Hay Al Farah,1,NaN,58,1,2,1,NaN,SERVICE CHERKI met en vente un appartement d'u...
33318,garçonnière,180 000 DH,Casablanca,"Appartements, à vendre",Ben M'sick,4,NaN,NaN,1,1,1,NaN,garçonnière au quatrième étage. tetri
33319,شقة مجهزة,125 000 DH,Agadir,"Appartements, à vendre",Hay Salam,1,6-10 ans,122,1,3,2,1.0,شقة مجهزة للبيع حي السلام بالقرب من جميع المرا...
